In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
#in and out data
dimension = 90
context_frames = 100

x = tf.placeholder(tf.float32,[None,dimension,context_frames]) #inputs
y = tf.placeholder(tf.float32,[None,2]) #labels

In [3]:
#conv layer 1
#[filter_height, filter_width, in-channels, out-channels]
weights_conv1 = tf.Variable(tf.truncated_normal(shape = [3,3,1,128], stddev = 0.01, dtype = tf.float32))
bias_conv1 = tf.Variable(tf.constant(0.1,shape = [128]))

# reshape of input -1 dynamically calculates the batch_size,in_height,in_width,in_channels
input_x = tf.reshape(x,[-1,dimension,context_frames,1])
print(input_x)
conv1 = tf.nn.conv2d(input_x,weights_conv1, strides = [1,2,2,1],padding = 'VALID')
conv1_out = tf.nn.relu(conv1+bias_conv1)
print(conv1_out)

Tensor("Reshape:0", shape=(?, 90, 100, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 44, 49, 128), dtype=float32)


In [4]:
#conv layer 2
weights_conv2 = tf.Variable(tf.truncated_normal(shape = [3,3,128,64], stddev = 0.01, dtype = tf.float32))
bias_conv2 = tf.Variable(tf.constant(0.1,shape = [64]))

conv2 = tf.nn.conv2d(conv1_out,weights_conv2,strides = [1,2,2,1], padding = 'VALID')
conv2_out = tf.nn.relu(conv2+bias_conv2)
print(conv2_out)

Tensor("Relu_1:0", shape=(?, 21, 24, 64), dtype=float32)


In [5]:
#conv layer 3
weights_conv3 = tf.Variable(tf.truncated_normal(shape = [5,5,64,64], stddev = 0.01, dtype = tf.float32))
bias_conv3 = tf.Variable(tf.constant(0.1,shape = [64]))

conv3 = tf.nn.conv2d(conv2_out,weights_conv3,strides = [1,1,1,1], padding = 'VALID')
conv3_out = tf.nn.relu(conv3+bias_conv3)
print(conv3_out)

Tensor("Relu_2:0", shape=(?, 17, 20, 64), dtype=float32)


In [6]:
#max pool 1
pool1_out = tf.layers.max_pooling2d(conv3_out, pool_size = [4,4], strides = [1,1], padding = 'VALID')
#conv4_out = tf.nn.relu(conv4+bias_conv4)
print(pool1_out)


Tensor("max_pooling2d/MaxPool:0", shape=(?, 14, 17, 64), dtype=float32)


In [7]:
#max pool 2
pool2_out = tf.layers.max_pooling2d(pool1_out, pool_size = [6,6], strides = [1,1], padding = 'VALID')
#conv4_out = tf.nn.relu(conv4+bias_conv4)
print(pool2_out)

Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 9, 12, 64), dtype=float32)


In [8]:
#print(pool_out.shape[1])
reduced_dim = pool2_out.shape[1] * pool2_out.shape[2] * pool2_out.shape[3]
print(type(reduced_dim),int(reduced_dim))

#dense layer 1
weights_dense1 = tf.Variable(tf.truncated_normal([int(reduced_dim),256], stddev = 0.01, dtype = tf.float32))
bias_dense1 = tf.Variable(tf.constant(0.1,shape = [256]))

input_dense = tf.reshape(pool2_out, shape = [-1,int(reduced_dim)])
#print(input_dense)
dense1_sum = tf.matmul(input_dense,weights_dense1)+bias_dense1
dense1_output = tf.nn.relu(dense1_sum)
print(dense1_output)

<class 'tensorflow.python.framework.tensor_shape.Dimension'> 6912
Tensor("Relu_3:0", shape=(?, 256), dtype=float32)


In [9]:
#adding dropout to dense layer 1
keep_prob = tf.placeholder(dtype = tf.float32)
dense1_dropout = tf.nn.dropout(dense1_output,keep_prob)
print(dense1_dropout)

Tensor("dropout/mul:0", shape=(?, 256), dtype=float32)


In [10]:
# dense layer 2
weights_dense2 = tf.Variable(tf.truncated_normal([256,256], stddev = 0.01, dtype = tf.float32))
bias_dense2 = tf.Variable(tf.constant(0.1,shape = [256]))

dense2_sum = tf.matmul(dense1_dropout,weights_dense2)+ bias_dense2
dense2_output = tf.nn.relu(dense2_sum)
print(dense2_output)

Tensor("Relu_4:0", shape=(?, 256), dtype=float32)


In [11]:
dense2_dropout = tf.nn.dropout(dense2_output,keep_prob)
print(dense2_dropout)

Tensor("dropout_1/mul:0", shape=(?, 256), dtype=float32)


In [12]:
#output layer
output_weights = tf.Variable(tf.truncated_normal(shape = [256,2], stddev = 0.01, dtype = tf.float32))
output_bias = tf.Variable(tf.constant(0.1,shape = [2]))

output_sum = tf.matmul(dense2_dropout,output_weights)+output_bias
final_output = tf.nn.softmax(output_sum)
print(final_output)

Tensor("Softmax:0", shape=(?, 2), dtype=float32)


In [13]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = final_output))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(final_output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [14]:
#initializing tensors using session

session = tf.InteractiveSession()

init = tf.global_variables_initializer()
#saver = tf.train.Saver(max_to_keep = None)



In [15]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [16]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    dev_num = len(list(os.listdir(test_path)))//2
    
    for j in range(1, dev_num+1):
        filename = "features_"+str(j)+".npy"
        labelsname= "labels_"+str(j)+".npy"
        
        features = np.load(os.path.join(test_path, filename))
        labels = np.load(os.path.join(test_path, labelsname))
        patches = labels.shape[0]
        patch_prob = session.run(final_output, feed_dict = {x:features, keep_prob : 1.0})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [19]:
#Training and testing accuracy
saver = tf.train.Saver(max_to_keep=None)
session.run(init)

features_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\overlapData_90D"
development_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\devoverlapData_90D"
#evaluation_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evaloverlapData_90D"
model_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Models_90D_CNN_3c2p2d"
    
dev_sp_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\dev_sp_90D_CNN_3c2p2d"
#eva_sp_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\eval_sp_90D_CNN_3c2p2d" 

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    

    

In [18]:
print(batches)

255


In [20]:
for i in range(1,50):
    
    for j in range(1,batches+1):  #number of batches


        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"

        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        #print("before  ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))
        train_step.run(session = session,feed_dict={ x:features, y:labels, keep_prob:0.5})
        #print("after   ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))


    save_path = saver.save(session,os.path.join(model_path,"Model_"+str(i+1)))

    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()
    
    

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_2 is not in all_model_checkpoint_paths. Manually adding it.
iteration   2
FRR   0.4  FAR    0.4    mid    0.170349825793
 develop eer   0.4

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_3 is not in all_model_checkpoint_paths. Manually adding it.
iteration   3
FRR   0.36578947368421055  FAR    0.3663157894736842    mid    0.189528179665
 develop eer   0.36578947368421055

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_4 is not in all_model_checkpoint_paths. Manually adding it.
iteration   4
FRR   0.3592105263157895  FAR    0.36    mid    0.147019771969
 develop eer   0.3592105263157895

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_5 is not in all_model_checkpoint_paths. Manually adding it.
iteration   5
FRR   0.3657894736842

iteration   30
FRR   0.14736842105263157  FAR    0.14736842105263157    mid    1.03887076601
 develop eer   0.14736842105263157

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_31 is not in all_model_checkpoint_paths. Manually adding it.
iteration   31
FRR   0.15394736842105264  FAR    0.15473684210526314    mid    -0.939066932229
 develop eer   0.15394736842105264

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_32 is not in all_model_checkpoint_paths. Manually adding it.
iteration   32
FRR   0.14868421052631578  FAR    0.14947368421052631    mid    -1.10337876423
 develop eer   0.14868421052631578

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_33 is not in all_model_checkpoint_paths. Manually adding it.
iteration   33
FRR   0.14868421052631578  FAR    0.14947368421052631    mid    -1.23602609049
 develop eer   0.1486842105263

In [21]:

with tf.Session() as session:
    saver.restore(session,os.path.join(model_path,"Model_"+str(50)))
    
    for i in range(50,75):
        for j in range(1,batches+1):
            
            filename = "batch_"+str(j)+".npy"
            labelsname= "label_"+str(j)+".npy"

            features = np.load(os.path.join(features_path, filename))
            labels = np.load(os.path.join(features_path, labelsname))
            
            train_step.run(session = session,feed_dict={ x:features, y:labels, keep_prob:0.5})
        save_path = saver.save(session,os.path.join(model_path,"Model_"+str(i+1)))
        
        print(" iteration   ", i+1)
        eer_dev = get_EER(development_path, (i+1), dev_sp_path)
        print(" develop eer"    , eer_dev)
        print()
        
        

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_50
INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_51 is not in all_model_checkpoint_paths. Manually adding it.
 iteration    51
FRR   0.14210526315789473  FAR    0.1431578947368421    mid    0.346311419948
 develop eer 0.14210526315789473

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_52 is not in all_model_checkpoint_paths. Manually adding it.
 iteration    52
FRR   0.14736842105263157  FAR    0.14736842105263157    mid    -0.724397556622
 develop eer 0.14736842105263157

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_53 is not in all_model_checkpoint_paths. Manually adding it.
 iteration    53
FRR   0.14473684210526316  FAR    0.14526315789473684    mid    -1.08741538689
 develop eer 0.1447368421052

In [22]:
evaluation_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evaloverlapData_90D"
eva_sp_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\eval_sp_90D_CNN_3c2p2d" 

if not os.path.exists(eva_sp_path):
    os.makedirs(eva_sp_path)
    
with tf.Session() as session:
    for i in range(1,50):
        saver.restore(session,os.path.join(model_path,"Model_"+str(i+1)))
        print("iteration ", (i+1))
        eval_err = get_EER(evaluation_path, (i+1), eva_sp_path)
        print(" evaluation err  ", eval_err)
        print()

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_2
iteration  2
FRR   0.44838212634822805  FAR    0.4484510326449034    mid    0.170382076786
 evaluation err   0.44838212634822805

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_3
iteration  3
FRR   0.42295839753466874  FAR    0.42371752165223187    mid    0.189829950531
 evaluation err   0.42295839753466874

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_4
iteration  4
FRR   0.42449922958397535  FAR    0.42513324450366424    mid    0.15734212368
 evaluation err   0.42449922958397535

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_5
iteration  5
FRR   0.43297380585516176  FAR    0.43312791472351764    mid    0.332366216363
 

iteration  34
FRR   0.19645608628659475  FAR    0.1966189207195203    mid    -1.29245051382
 evaluation err   0.19645608628659475

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_35
iteration  35
FRR   0.19568567026194145  FAR    0.1963690872751499    mid    0.0913331768129
 evaluation err   0.19568567026194145

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_36
iteration  36
FRR   0.20184899845916796  FAR    0.20186542305129915    mid    1.42588151537
 evaluation err   0.20184899845916796

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNN_3c2p2d\Model_37
iteration  37
FRR   0.19799691833590138  FAR    0.19795136575616257    mid    2.01229275819
 evaluation err   0.19799691833590138

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof20

In [11]:
import os
import numpy as np
development_path = "E:\\Aditya\\ASVspoof2019_CFCC_IF_CMN\\78D\\overlapData_78D"
dev_num = len(os.listdir(development_path))

In [6]:
# features_path = "E:\\Aditya\\ASVspoof2017_CFCC_IF_CMVN\\devoverlapData_117D"
# batches = (len(list(os.listdir(features_path))))//2
# print(batches)

In [13]:
for i in range(1,dev_num):
    filename = 'batch_'+str(i)+'.npy'
    file = np.load(os.path.join(development_path,filename))
    if(np.sum(np.isnan(file)) > 0) :
        print(filename)
    

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Aditya\\ASVspoof2019_CFCC_IF_CMN\\78D\\overlapData_78D\\batch_127.npy'

In [9]:
a = [1,np.log(-1),np.log(5/6)]
print(np.isnan(a).any())

True


C:\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
